In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-41c9e93ae06c>:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
from google.colab import files
uploaded  = files.upload()

Saving excel.xlsx to excel.xlsx


In [ ]:
import io 
df = pd.read_excel(io.BytesIO(uploaded['excel.xlsx']), header=None, names=['Data'])
df.head()

,Data
0,Go.\tजा.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #3138228 (sabretou)
1,Run!\tपळ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138217 (sabretou)
2,Run!\tधाव!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138218 (sabretou)
3,Run!\tपळा!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138219 (sabretou)
4,Run!\tधावा!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138220 (sabretou)


In [ ]:
df['Data'].str.split('\t', expand=True)

,0,1,2
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #3138228 (sabretou)
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138217 (sabretou)
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138218 (sabretou)
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138219 (sabretou)
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #3138220 (sabretou)
...,...,...,...
39956,Just saying you don't like fish because of the bones is not really a good reason for not liking fish.,हड्डींमुळे मासे आवडत नाही असं म्हणणं हे काय मासे नावडण्यासाठी चांगलं कारण नाहीये.,CC-BY 2.0 (France) Attribution: tatoeba.org #1190099 (CK) & #2271250 (sabretou)
39957,The Japanese Parliament today officially elected Ryoutarou Hashimoto as the country's 52nd prime minister.,आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ हाशिमोतो यांना देशाचे ५२ावे पंतप्रधान म्हणून निवडले.,CC-BY 2.0 (France) Attribution: tatoeba.org #281385 (BraveSentry) & #8352587 (sabretou)
39958,"Tom tried to sell his old VCR instead of throwing it away, but no one would buy it, so he ended up throwing it away.","टॉमने त्याचा जुना व्ही.सी.आर फेकून टाकण्याऐवजी विकून टाकण्याचा प्रयत्न केला, पण विकत घ्यायला कोणीच तयार नव्हतं, म्हणून त्याला तो शेवटी फेकूनच टाकायला लागला.",CC-BY 2.0 (France) Attribution: tatoeba.org #1092619 (CK) & #2521453 (sabretou)
39959,"You can't view Flash content on an iPad. However, you can easily email yourself the URLs of these web pages and view that content on your regular computer when you get home.",आयपॅडवर फ्लॅश आशय बघता येत नाही. पण तुम्ही त्या वेब पानांचे यूआरएल स्वतःला ईमेल करून तोच आशय घरी पोहोचल्यावर आपल्या रोजच्या संगणकावर पाहू शकता.,CC-BY 2.0 (France) Attribution: tatoeba.org #918414 (CK) & #2386247 (sabretou)


In [ ]:
df['Marathi'] = df['Data'].str.split('\t', expand=True)[1]
df['English'] = df['Data'].str.split('\t', expand=True)[0]

In [ ]:
df = df.drop(['Data'], axis=1)

In [ ]:
# df = df[:30000]

In [ ]:
df.head()

,Marathi,English
0,जा.,Go.
1,पळ!,Run!
2,धाव!,Run!
3,पळा!,Run!
4,धावा!,Run!


In [ ]:
df.shape

(39961, 2)

In [ ]:
# Lowercase all characters
df['English']=df['English'].apply(lambda x: str(x))
df['Marathi']=df['Marathi'].apply(lambda x: str(x))
df['English']=df['English'].apply(lambda x: x.lower())
df['Marathi']=df['Marathi'].apply(lambda x: x.lower())

In [ ]:
# Import stopwords with nltk.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['English']=df['English'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['Marathi']=df['Marathi'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
# Remove quotes
df['English']=df['English'].apply(lambda x: re.sub("'", '', x))
df['Marathi']=df['Marathi'].apply(lambda x: re.sub("'", '', x))

In [ ]:
df.head()

,Marathi,English
0,जा.,go.
1,पळ!,run!
2,धाव!,run!
3,पळा!,run!
4,धावा!,run!


In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df['English']=df['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['Marathi']=df['Marathi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
df.head()

,Marathi,English
0,जा,go
1,पळ,run
2,धाव,run
3,पळा,run
4,धावा,run


In [ ]:
df.tail()

,Marathi,English
39956,हड्डींमुळे मासे आवडत नाही असं म्हणणं हे काय मासे नावडण्यासाठी चांगलं कारण नाहीये,saying like fish bones really good reason liking fish
39957,आज जपानी संसदेने अधिकृतरित्या र्‍यौतारौ हाशिमोतो यांना देशाचे ५२ावे पंतप्रधान म्हणून निवडले,japanese parliament today officially elected ryoutarou hashimoto countrys 52nd prime minister
39958,टॉमने त्याचा जुना व्हीसीआर फेकून टाकण्याऐवजी विकून टाकण्याचा प्रयत्न केला पण विकत घ्यायला कोणीच तयार नव्हतं म्हणून त्याला तो शेवटी फेकूनच टाकायला लागला,tom tried sell old vcr instead throwing away one would buy it ended throwing away
39959,आयपॅडवर फ्लॅश आशय बघता येत नाही पण तुम्ही त्या वेब पानांचे यूआरएल स्वतःला ईमेल करून तोच आशय घरी पोहोचल्यावर आपल्या रोजच्या संगणकावर पाहू शकता,cant view flash content ipad however easily email urls web pages view content regular computer get home
39960,१९६९मध्ये रॉजर मिलरने यू डोन्ट वॉन्ट माय लव्ह नावाचं गाणं रेकॉर्ड केलं आज हे गाणं इन द समर टाइम म्हणून जास्त प्रसिद्ध आहे त्याने लिहिलेलं व गायलेलं हे असं पहिलं गाणं होतं जे लोकप्रिय झालं,1969 roger miller recorded song called you want love today song better known in summer time first song wrote sang became popular


In [ ]:
remove_digits = str.maketrans('', '', digits)

In [ ]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [ ]:
a = df['English'][0].translate(remove_digits)

In [ ]:
a

'go'

In [ ]:
a.strip()

'go'

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['English']=df['English'].apply(lambda x: x.translate(remove_digits))
df['Marathi']=df['Marathi'].apply(lambda x: x.translate(remove_digits))

df['Marathi'] = df['Marathi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [ ]:
# Remove extra spaces
df['English']=df['English'].apply(lambda x: x.strip())
df['Marathi']=df['Marathi'].apply(lambda x: x.strip())
df['English']=df['English'].apply(lambda x: re.sub(" +", " ", x))
df['Marathi']=df['Marathi'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add start and end tokens to target sequences
df['English'] = df['English'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
df['English'][0]

'START_ go _END'

In [ ]:
### Get English Vocabulary
all_eng_words=set()
for eng in df['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
all_eng_words

{'halloweens',
 'james',
 'youre',
 'banana',
 'economic',
 'surgeon',
 'score',
 'gaga',
 'apology',
 'two',
 'donated',
 'lays',
 'bury',
 'management',
 'pancakes',
 'syria',
 'lines',
 'applied',
 'closely',
 'relax',
 'crash',
 'find',
 'brunettes',
 'someones',
 'sheriff',
 'sell',
 'cheesecake',
 'introduced',
 'heroine',
 'gamble',
 'palace',
 'champion',
 'sumter',
 'teachers',
 'dropping',
 'counting',
 'president',
 'danzig',
 'republican',
 'software',
 'st',
 'painting',
 'accident',
 'die',
 'lifted',
 'puzzles',
 'meets',
 'donate',
 'yawned',
 'gramophone',
 'schizophrenic',
 'permission',
 'pens',
 'salad',
 'vomited',
 'incorrectly',
 'eighteenth',
 'unarmed',
 'climbing',
 'lunatic',
 'receive',
 'editor',
 'commented',
 'envelope',
 'kissing',
 'pity',
 'fiction',
 'jokes',
 'towns',
 'pastor',
 'navy',
 'utah',
 'kilo',
 'elk',
 'languages',
 'snoring',
 'afraid',
 'headset',
 'till',
 'draw',
 'sixteen',
 'paying',
 'industrialized',
 'fact',
 'blew',
 'destroy',


In [ ]:
### Get Marathi Vocabulary
all_marathi_words=set()
for mar in df['Marathi']:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [ ]:
# li = list(all_marathi_words)

In [ ]:
# li

In [ ]:
# li.sort()

In [ ]:
# li.pop(0)

In [ ]:
# li

In [ ]:
# all_marathi_words=set(li)

In [ ]:
# all_marathi_words

In [ ]:
df['length_eng']=df['English'].apply(lambda x:len(x.split(" ")))
df['length_mar']=df['Marathi'].apply(lambda x:len(x.split(" ")))

In [ ]:
df['length_eng']

0        3 
1        3 
2        3 
3        3 
4        3 
        .. 
39956    11
39957    13
39958    17
39959    19
39960    23
Name: length_eng, Length: 39961, dtype: int64

In [ ]:
df.head()

,Marathi,English,length_eng,length_mar
0,जा,START_ go _END,3,1
1,पळ,START_ run _END,3,1
2,धाव,START_ run _END,3,1
3,पळा,START_ run _END,3,1
4,धावा,START_ run _END,3,1


In [ ]:
df['length_mar']

0        1 
1        1 
2        1 
3        1 
4        1 
        .. 
39956    13
39957    12
39958    23
39959    24
39960    35
Name: length_mar, Length: 39961, dtype: int64

In [ ]:
df[df['length_eng']>30].shape

(0, 4)

In [ ]:
df[df['length_mar']>30].shape

(1, 4)

In [ ]:
df=df[df['length_eng']<=20]
df=df[df['length_mar']<=20]

In [ ]:
print("maximum length of Marathi Sentence ",max(df['length_mar']))
print("maximum length of English Sentence ",max(df['length_eng']))

maximum length of Marathi Sentence  35
maximum length of English Sentence  23


In [ ]:
max_length_src=max(df['length_mar'])
max_length_tar=max(df['length_eng'])

In [ ]:
print(max_length_src)

35


In [ ]:
print(max_length_tar)

13


In [ ]:
input_words = sorted(list(all_marathi_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_marathi_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(13608, 5626)

In [ ]:
input_words

['b',
 'ds',
 'h₂o',
 'ntt',
 'tatoebaorg',
 'uk',
 'अ',
 'अँकरेजमार्गे',
 'अँजिलीस',
 'अँटीव्हायरस',
 'अँड',
 'अँडी',
 'अँडीजमधल्या',
 'अँब्युलन्स',
 'अंक',
 'अंकांनी',
 'अंकारा',
 'अंकारात',
 'अंकारामध्ये',
 'अंग',
 'अंगठ्या',
 'अंगाला',
 'अंगोला',
 'अंघोळ',
 'अंठ्यांपेक्षा',
 'अंड',
 'अंडरवुडचा',
 'अंडरवेअर',
 'अंडी',
 'अंडींचा',
 'अंडीसुद्धा',
 'अंड्यांवर',
 'अंत',
 'अंतर',
 'अंतराळयात्री',
 'अंतराळयान',
 'अंतराळयानाचं',
 'अंतराळयानातून',
 'अंतराळवीर',
 'अंतराळात',
 'अंतरित',
 'अंतिम',
 'अंदाज',
 'अंदाजे',
 'अंधार',
 'अंधाराची',
 'अंधारात',
 'अंमलात',
 'अंशांकित',
 'अकरा',
 'अकरावीत',
 'अकल्पनीय',
 'अकाउंट',
 'अकाउंटमध्ये',
 'अकॉर्डियन',
 'अक्कलदाढ',
 'अक्वॅरियममध्ये',
 'अक्षर',
 'अक्षरं',
 'अक्षरांनी',
 'अक्षरांमध्ये',
 'अक्षरे',
 'अखाद्य',
 'अखिलेउस',
 'अख्खं',
 'अख्खा',
 'अख्ख्या',
 'अगं',
 'अगदी',
 'अगोदर',
 'अगोदरच',
 'अचानक',
 'अचानकच',
 'अचूक',
 'अच्छा',
 'अजिबात',
 'अजिबातच',
 'अजीबात',
 'अजुनपर्यंत',
 'अजुनही',
 'अजून',
 'अजूनच',
 'अजूनपर्यंत',
 'अजूनही',
 'अज्ञात',
 'अट',

In [ ]:
target_words

['START_',
 '_END',
 'a',
 'abbreviation',
 'ability',
 'able',
 'about',
 'above',
 'abroad',
 'absent',
 'absolute',
 'absolutely',
 'accent',
 'accept',
 'acceptable',
 'accepted',
 'access',
 'accessible',
 'accident',
 'accidentally',
 'accomplices',
 'according',
 'accordion',
 'account',
 'accused',
 'accusing',
 'accustomed',
 'ace',
 'aces',
 'ache',
 'aches',
 'achille',
 'achilles',
 'acid',
 'acknowledged',
 'acquired',
 'acres',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'activist',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'add',
 'added',
 'addict',
 'addicted',
 'addition',
 'additional',
 'addon',
 'address',
 'administrative',
 'admire',
 'admit',
 'admitted',
 'adopt',
 'adopted',
 'ads',
 'adultery',
 'advance',
 'advantage',
 'adverb',
 'advertisements',
 'advice',
 'advised',
 'adviser',
 'advisers',
 'advisor',
 'affair',
 'affairs',
 'afford',
 'afghanistan',
 'afraid',
 'africa',
 'african',
 'africans',
 'after',
 'afternoon',

In [ ]:
num_decoder_tokens

5626

In [ ]:
num_decoder_tokens += 1

In [ ]:
num_decoder_tokens

5627

In [ ]:
num_encoder_tokens

13608

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
input_token_index

{'b': 1,
 'ds': 2,
 'h₂o': 3,
 'ntt': 4,
 'tatoebaorg': 5,
 'uk': 6,
 'अ': 7,
 'अँकरेजमार्गे': 8,
 'अँजिलीस': 9,
 'अँटीव्हायरस': 10,
 'अँड': 11,
 'अँडी': 12,
 'अँडीजमधल्या': 13,
 'अँब्युलन्स': 14,
 'अंक': 15,
 'अंकांनी': 16,
 'अंकारा': 17,
 'अंकारात': 18,
 'अंकारामध्ये': 19,
 'अंग': 20,
 'अंगठ्या': 21,
 'अंगाला': 22,
 'अंगोला': 23,
 'अंघोळ': 24,
 'अंठ्यांपेक्षा': 25,
 'अंड': 26,
 'अंडरवुडचा': 27,
 'अंडरवेअर': 28,
 'अंडी': 29,
 'अंडींचा': 30,
 'अंडीसुद्धा': 31,
 'अंड्यांवर': 32,
 'अंत': 33,
 'अंतर': 34,
 'अंतराळयात्री': 35,
 'अंतराळयान': 36,
 'अंतराळयानाचं': 37,
 'अंतराळयानातून': 38,
 'अंतराळवीर': 39,
 'अंतराळात': 40,
 'अंतरित': 41,
 'अंतिम': 42,
 'अंदाज': 43,
 'अंदाजे': 44,
 'अंधार': 45,
 'अंधाराची': 46,
 'अंधारात': 47,
 'अंमलात': 48,
 'अंशांकित': 49,
 'अकरा': 50,
 'अकरावीत': 51,
 'अकल्पनीय': 52,
 'अकाउंट': 53,
 'अकाउंटमध्ये': 54,
 'अकॉर्डियन': 55,
 'अक्कलदाढ': 56,
 'अक्वॅरियममध्ये': 57,
 'अक्षर': 58,
 'अक्षरं': 59,
 'अक्षरांनी': 60,
 'अक्षरांमध्ये': 61,
 'अक्षरे': 62,
 'अखाद्य': 63,
 

In [ ]:
target_token_index

{'START_': 1,
 '_END': 2,
 'a': 3,
 'abbreviation': 4,
 'ability': 5,
 'able': 6,
 'about': 7,
 'above': 8,
 'abroad': 9,
 'absent': 10,
 'absolute': 11,
 'absolutely': 12,
 'accent': 13,
 'accept': 14,
 'acceptable': 15,
 'accepted': 16,
 'access': 17,
 'accessible': 18,
 'accident': 19,
 'accidentally': 20,
 'accomplices': 21,
 'according': 22,
 'accordion': 23,
 'account': 24,
 'accused': 25,
 'accusing': 26,
 'accustomed': 27,
 'ace': 28,
 'aces': 29,
 'ache': 30,
 'aches': 31,
 'achille': 32,
 'achilles': 33,
 'acid': 34,
 'acknowledged': 35,
 'acquired': 36,
 'acres': 37,
 'across': 38,
 'act': 39,
 'acted': 40,
 'acting': 41,
 'action': 42,
 'activist': 43,
 'actor': 44,
 'actors': 45,
 'actress': 46,
 'acts': 47,
 'actual': 48,
 'actually': 49,
 'add': 50,
 'added': 51,
 'addict': 52,
 'addicted': 53,
 'addition': 54,
 'additional': 55,
 'addon': 56,
 'address': 57,
 'administrative': 58,
 'admire': 59,
 'admit': 60,
 'admitted': 61,
 'adopt': 62,
 'adopted': 63,
 'ads': 64,
 '

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
reverse_input_char_index

{1: 'b',
 2: 'ds',
 3: 'h₂o',
 4: 'ntt',
 5: 'tatoebaorg',
 6: 'uk',
 7: 'अ',
 8: 'अँकरेजमार्गे',
 9: 'अँजिलीस',
 10: 'अँटीव्हायरस',
 11: 'अँड',
 12: 'अँडी',
 13: 'अँडीजमधल्या',
 14: 'अँब्युलन्स',
 15: 'अंक',
 16: 'अंकांनी',
 17: 'अंकारा',
 18: 'अंकारात',
 19: 'अंकारामध्ये',
 20: 'अंग',
 21: 'अंगठ्या',
 22: 'अंगाला',
 23: 'अंगोला',
 24: 'अंघोळ',
 25: 'अंठ्यांपेक्षा',
 26: 'अंड',
 27: 'अंडरवुडचा',
 28: 'अंडरवेअर',
 29: 'अंडी',
 30: 'अंडींचा',
 31: 'अंडीसुद्धा',
 32: 'अंड्यांवर',
 33: 'अंत',
 34: 'अंतर',
 35: 'अंतराळयात्री',
 36: 'अंतराळयान',
 37: 'अंतराळयानाचं',
 38: 'अंतराळयानातून',
 39: 'अंतराळवीर',
 40: 'अंतराळात',
 41: 'अंतरित',
 42: 'अंतिम',
 43: 'अंदाज',
 44: 'अंदाजे',
 45: 'अंधार',
 46: 'अंधाराची',
 47: 'अंधारात',
 48: 'अंमलात',
 49: 'अंशांकित',
 50: 'अकरा',
 51: 'अकरावीत',
 52: 'अकल्पनीय',
 53: 'अकाउंट',
 54: 'अकाउंटमध्ये',
 55: 'अकॉर्डियन',
 56: 'अक्कलदाढ',
 57: 'अक्वॅरियममध्ये',
 58: 'अक्षर',
 59: 'अक्षरं',
 60: 'अक्षरांनी',
 61: 'अक्षरांमध्ये',
 62: 'अक्षरे',
 63: 'अखाद्य',
 

In [ ]:
reverse_target_char_index

{1: 'START_',
 2: '_END',
 3: 'a',
 4: 'abbreviation',
 5: 'ability',
 6: 'able',
 7: 'about',
 8: 'above',
 9: 'abroad',
 10: 'absent',
 11: 'absolute',
 12: 'absolutely',
 13: 'accent',
 14: 'accept',
 15: 'acceptable',
 16: 'accepted',
 17: 'access',
 18: 'accessible',
 19: 'accident',
 20: 'accidentally',
 21: 'accomplices',
 22: 'according',
 23: 'accordion',
 24: 'account',
 25: 'accused',
 26: 'accusing',
 27: 'accustomed',
 28: 'ace',
 29: 'aces',
 30: 'ache',
 31: 'aches',
 32: 'achille',
 33: 'achilles',
 34: 'acid',
 35: 'acknowledged',
 36: 'acquired',
 37: 'acres',
 38: 'across',
 39: 'act',
 40: 'acted',
 41: 'acting',
 42: 'action',
 43: 'activist',
 44: 'actor',
 45: 'actors',
 46: 'actress',
 47: 'acts',
 48: 'actual',
 49: 'actually',
 50: 'add',
 51: 'added',
 52: 'addict',
 53: 'addicted',
 54: 'addition',
 55: 'additional',
 56: 'addon',
 57: 'address',
 58: 'administrative',
 59: 'admire',
 60: 'admit',
 61: 'admitted',
 62: 'adopt',
 63: 'adopted',
 64: 'ads',
 6

In [ ]:
df.head(15)

,Marathi,English,length_eng,length_mar
0,जा,START_ go _END,3,1
1,पळ,START_ run _END,3,1
2,धाव,START_ run _END,3,1
3,पळा,START_ run _END,3,1
4,धावा,START_ run _END,3,1
5,कोण,START_ who _END,3,1
6,वाह,START_ wow _END,3,1
7,आग,START_ fire _END,3,1
8,फायर,START_ fire _END,3,1
9,वाचवा,START_ help _END,3,1


In [ ]:
from sklearn.model_selection import train_test_split
X, y = df['Marathi'], df['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((31968,), (7993,))

In [ ]:
X_train

12290    टॉम घाईघाईत निघाला                                       
34027    मला रडायचं असतं तेव्हा मी तुझा विचार करतो                
33922    टॉमने काही वर्ष फ्रेंचचा अभ्यास केला                     
7954     बस येत आहे                                               
12403    स्वतः करून पाहा                                          
              ...                                                 
6265     कुत्रा मेलेला                                            
11284    मी खेळ जिंकलेय                                           
38158    त्याच्या आजोबांनी त्याच्यासाठी एक महागडं खेळणं विकत घेतलं
860      त्याच्याकडे वाईन आहे                                     
15795    शनिवारची रात्र होती                                      
Name: Marathi, Length: 31968, dtype: object

In [ ]:
y_train

12290    START_ tom left hurry _END                  
34027    START_ want cry think you _END              
33922    START_ tom studied french years _END        
7954     START_ bus coming _END                      
12403    START_ try yourself _END                    
                   ...                               
6265     START_ dog dead _END                        
11284    START_ game _END                            
38158    START_ grandfather bought expensive toy _END
860      START_ wine _END                            
15795    START_ saturday night _END                  
Name: English, Length: 31968, dtype: object

In [ ]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    4082700     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    1688400     ['input_2[0][0]']                
                                                                                              

In [ ]:
model.save('mar-to-eng.h5')

In [ ]:
a, b = next(generate_batch())

In [ ]:
a

[array([[ 5608.,   670.,  8473., ...,     0.,     0.,     0.],
        [  623.,  3058., 12838., ...,     0.,     0.,     0.],
        [ 9619.,  4783.,  6244., ...,     0.,     0.,     0.],
        ...,
        [ 5513.,  4019., 12499., ...,     0.,     0.,     0.],
        [ 5483.,  9909.,  4532., ...,     0.,     0.,     0.],
        [ 5543.,  9672.,  6401., ...,     0.,     0.,     0.]],
       dtype=float32),
 array([[1.000e+00, 5.040e+03, 3.524e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 4.655e+03, 4.442e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 2.473e+03, 5.390e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [1.000e+00, 3.442e+03, 2.171e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 2.185e+03, 5.249e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.000e+00, 4.568e+03, 3.265e+03, ..., 0.000e+00, 0.000e+00,
         0.000e+00]], dtype=float32)]

In [ ]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
X_train[100]

'मी पण'

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

<ipython-input-68-029ac434b37f>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/10
499/499 [==============================] - 622s 1s/step - loss: 0.1935 - accuracy: 0.9664 - val_loss: 1.8942 - val_accuracy: 0.7308
Epoch 2/10
499/499 [==============================] - 579s 1s/step - loss: 0.1438 - accuracy: 0.9759 - val_loss: 1.9026 - val_accuracy: 0.7360
Epoch 3/10
499/499 [==============================] - 646s 1s/step - loss: 0.1106 - accuracy: 0.9816 - val_loss: 1.9269 - val_accuracy: 0.7382
Epoch 4/10
499/499 [==============================] - 632s 1s/step - loss: 0.0868 - accuracy: 0.9859 - val_loss: 1.9407 - val_accuracy: 0.7408
Epoch 5/10
499/499 [==============================] - 588s 1s/step - loss: 0.0694 - accuracy: 0.9886 - val_loss: 1.9597 - val_accuracy: 0.7420
Epoch 6/10
499/499 [==============================] - 593s 1s/step - loss: 0.0590 - accuracy: 0.9896 - val_loss: 1.9870 - val_accuracy: 0.7440
Epoch 7/10
499/499 [==============================] - 590s 1s/step - loss: 0.0551 - accuracy: 0.9897 - val_loss: 2.0097 - val_accuracy: 0.7396

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 27ms/step
Input Marathi sentence: टॉम घाईघाईत निघाला
Actual English Translation:  tom left hurry 
Predicted English Translation:  tom left hurry 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 29ms/step
Input Marathi sentence: मला रडायचं असतं तेव्हा मी तुझा विचार करतो
Actual English Translation:  want cry think you 
Predicted English Translation:  want cry think you 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input Marathi sentence: मी टॉमवर नजर ठेवतोय
Actual English Translation:  im watching tom 
Predicted English Translation:  im watching tom 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 17ms/step
Input Marathi sentence: बियर कोणी आणली
Actual English Translation:  brought beer 
Predicted English Translation:  brought beer 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input Marathi sentence: माझ्याबरोबर खेळायचंय का
Actual English Translation:  want play me 
Predicted English Translation:  want play me 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input Marathi sentence: मजा आली
Actual English Translation:  fun 
Predicted English Translation:  fun 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 16ms/step
Input Marathi sentence: मी दर सकाळी साउअरक्राउट खाते
Actual English Translation:  eat sauerkraut every morning 
Predicted English Translation:  eat sauerkraut every morning 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input Marathi sentence: मला वाटतं मी तुमची मदत करू शकतो
Actual English Translation:  think help you 
Predicted English Translation:  think help you 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 33ms/step
Input Marathi sentence: आत या
Actual English Translation:  come inside 
Predicted English Translation:  come in 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print(decoded_sentence[:-4])
print('Predicted English Translation:', decoded_sentence[:-4]) 

1/1 [==============================] - 0s 42ms/step
Input Marathi sentence: बस येत आहे
Actual English Translation:  bus coming 
 bus coming 
Predicted English Translation:  bus coming 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
ref = [
    'मी पूर्णपणे मान्य करतोा'.split(),
    'पूर्णपणे मी मान्य करतोा'.split(),
    'पूर्णपणे मान्य मी करतोा'.split()
]
test = 'मी मान्य करतो'.split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))
 
test01 = 'पूर्णपणे मान्य करतो'.split()
print('BLEU score for test01-> {}'.format(sentence_bleu(ref, test01)))

BLEU score for test-> 8.121328445417258e-155
BLEU score for test01-> 8.121328445417258e-155


In [ ]:
%%bash
pip install numpy scipy librosa unidecode inflect librosa
apt-get update
apt-get install -y libsndfile1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,009 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-dri

In [ ]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn

Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0-2): 3 x Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (linear_layer): Linear(in_features=80, out_features=256, bias=False)
        )
        (1): LinearNorm(
          (linear_layer): Linear(in_features=256, out_features=256, bias=False)
        )
      )
    )
    (attention_rnn): LSTMCell(768, 1024)
    (attention_layer): Attention(
      (query_layer): LinearNorm(
        (linear_layer): Linear(in_features=1024, out_features=128, bias=False)
      )
      (memory_layer): LinearNorm(
        (linear_layer): Linear(in_fea

In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0-3): 4 x WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0-6): 7 x Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (7

Give Text Input here. 

In [ ]:
text = 'come inside'

In [ ]:
import torch
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils') 
sequences, lengths = utils.prepare_input_sequence([text])

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/text/__init__.py:74: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return s in _symbol_to_id and s is not '_' and s is not '~'


In [ ]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22056

NameError: ignored

In [ ]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

In [ ]:
from scipy.io.wavfile import write 
def generate_audio():
  return write("audio.wav", rate, audio_numpy)   
  
generate_audio

<function __main__.generate_audio()>

In [ ]:
from scipy.io.wavfile import write
write("audio.wav", rate, audio_numpy)

NameError: ignored

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display
def beep(): 
    display(Audio(audio_numpy,rate = rate)) 
    return 'hello'
beep();

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2

In [ ]:
import gradio as gr 
from IPython.display import Audio  
from IPython.core.display import display
#op = Audio(audio_numpy, rate=rate)  
iface = gr.Interface(beep,"text","text") 
iface.launch()

/usr/local/lib/python3.9/dist-packages/gradio/utils.py:875: UserWarning: Expected 0 arguments for function <function beep at 0x7fec2ad7ef70>, received 1.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/utils.py:883: UserWarning: Expected maximum 0 arguments for function <function beep at 0x7fec2ad7ef70>, received 1.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>